In [1]:
%pip install python-dotenv
%pip install -U langgraph langchain langchain-google-genai langchain-tavily python-dotenv aiosqlite

Note: you may need to restart the kernel to use updated packages.


   ---------------------------------------- 0.0/153.3 kB ? eta -:--:--
   ----- --------------------------------- 20.5/153.3 kB 640.0 kB/s eta 0:00:01
   ------------------ -------------------- 71.7/153.3 kB 975.2 kB/s eta 0:00:01
   ------------------------------------- -- 143.4/153.3 kB 1.2 MB/s eta 0:00:01
   ---------------------------------------- 153.3/153.3 kB 1.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/52.0 kB ? eta -:--:--
   ---------------------------------------- 52.0/52.0 kB 2.6 MB/s eta 0:00:00
  Attempting uninstall: langgraph-sdk
    Found existing installation: langgraph-sdk 0.2.0
    Uninstalling langgraph-sdk-0.2.0:
      Successfully uninstalled langgraph-sdk-0.2.0
  Attempting uninstall: langgraph
    Found existing installation: langgraph 0.6.5
    Uninstalling langgraph-0.6.5:
      Successfully uninstalled langgraph-0.6.5
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ['GOOGLE_API_KEY'] = os.getenv('GEMINI_API_KEY') 
os.environ['TAVILY_API_KEY'] = os.getenv('TAVILY_API_KEY')

In [2]:
import operator
from typing import Annotated, List, Any, Dict
from dataclasses import dataclass, field
from langgraph.graph import StateGraph, END
from langchain_core.messages import HumanMessage, SystemMessage, ToolMessage, BaseMessage, AnyMessage

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_tavily import TavilySearch

from langgraph.checkpoint.sqlite import SqliteSaver

from typing_extensions import TypedDict

import sqlite3

In [3]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

conn = sqlite3.connect("checkpoints.db", check_same_thread=False)
memory = SqliteSaver(conn)

In [4]:
class Agent:
    def __init__(self, model, tools, checkpointer, system=""):
        self.system = system
        
        graph = StateGraph(AgentState)

        graph.add_node("llm", self.call_gemini)

        graph.add_node("action", self.take_action)

        graph.add_conditional_edges("llm", self.exists_action, {True: "action", False: END})

        graph.add_edge("action", "llm")

        graph.set_entry_point("llm")

        self.graph = graph.compile(checkpointer=checkpointer)
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def call_gemini(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages

        print("Mensagens enviadas ao modelo:", messages)
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling Tool: {t['name']} with args: {t['args']}")
            result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Returning to LLM after action!")
        return {'messages': results}

In [5]:
current_tavily_api_key = os.getenv('TAVILY_API_KEY')
if not current_tavily_api_key:
    raise ValueError("TAVILY_API_KEY não encontrada. Certifique-se de que está no seu .env e python-dotenv está instalado.")

tool = TavilySearch(max_results=3, tavily_api_key=current_tavily_api_key)


prompt_system = """Você é um assistente de pesquisa inteligente. Use o mecanismo de busca (tavily_search_results_json) para procurar informações.
Você tem permissão para fazer múltiplas chamadas à ferramenta (em conjunto ou em sequência).
Busque informações apenas quando tiver certeza do que procurar.
Se precisar de mais detalhes para formular uma pergunta de acompanhamento, você tem permissão para fazer isso.
Quando solicitado a comparar informações (ex: qual é mais quente, maior, etc.), use as informações do histórico da conversa e dos resultados das ferramentas."""

model = ChatGoogleGenerativeAI(model="gemini-1.5-pro") 

abot = Agent(model, [tool], system=prompt_system, checkpointer=memory)

In [6]:
messages = [HumanMessage(content="Como está o tempo em São Paulo hoje (21/08/2025)?")]
thread = {"configurable": {"thread_id": "1"}} 


print("\n--- Pergunta 1: Tempo em São Paulo ---")
for event in abot.graph.stream({"messages": messages}, thread):
    for k, v in event.items():
        if k in ("llm", "action"): 
             print(f"{k}: {v['messages']}")


--- Pergunta 1: Tempo em São Paulo ---
Mensagens enviadas ao modelo: [SystemMessage(content='Você é um assistente de pesquisa inteligente. Use o mecanismo de busca (tavily_search_results_json) para procurar informações.\nVocê tem permissão para fazer múltiplas chamadas à ferramenta (em conjunto ou em sequência).\nBusque informações apenas quando tiver certeza do que procurar.\nSe precisar de mais detalhes para formular uma pergunta de acompanhamento, você tem permissão para fazer isso.\nQuando solicitado a comparar informações (ex: qual é mais quente, maior, etc.), use as informações do histórico da conversa e dos resultados das ferramentas.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Como está o tempo em São Paulo hoje?', additional_kwargs={}, response_metadata={}), AIMessage(content='Preciso de mais informações para responder à sua pergunta. Você poderia me dizer a data de hoje?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0

In [7]:
messages = [HumanMessage(content="E no Rio de Janeiro?")]
thread = {"configurable": {"thread_id": "1"}} 

print("\n--- Pergunta 2: Tempo no Rio de Janeiro ---")
for event in abot.graph.stream({"messages": messages}, thread):
    for k, v in event.items():
        if k in ("llm", "action"):
            print(f"{k}: {v['messages']}")


--- Pergunta 2: Tempo no Rio de Janeiro ---
Mensagens enviadas ao modelo: [SystemMessage(content='Você é um assistente de pesquisa inteligente. Use o mecanismo de busca (tavily_search_results_json) para procurar informações.\nVocê tem permissão para fazer múltiplas chamadas à ferramenta (em conjunto ou em sequência).\nBusque informações apenas quando tiver certeza do que procurar.\nSe precisar de mais detalhes para formular uma pergunta de acompanhamento, você tem permissão para fazer isso.\nQuando solicitado a comparar informações (ex: qual é mais quente, maior, etc.), use as informações do histórico da conversa e dos resultados das ferramentas.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Como está o tempo em São Paulo hoje?', additional_kwargs={}, response_metadata={}), AIMessage(content='Preciso de mais informações para responder à sua pergunta. Você poderia me dizer a data de hoje?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reaso

In [8]:
messages = [HumanMessage(content="Qual está mais quente?")]
thread = {"configurable": {"thread_id": "1"}} 

print("\n--- Pergunta 3: Comparação ---")
for event in abot.graph.stream({"messages": messages}, thread):
    for k, v in event.items():
        if k in ("llm", "action"):
            print(f"{k}: {v['messages']}")


--- Pergunta 3: Comparação ---
Mensagens enviadas ao modelo: [SystemMessage(content='Você é um assistente de pesquisa inteligente. Use o mecanismo de busca (tavily_search_results_json) para procurar informações.\nVocê tem permissão para fazer múltiplas chamadas à ferramenta (em conjunto ou em sequência).\nBusque informações apenas quando tiver certeza do que procurar.\nSe precisar de mais detalhes para formular uma pergunta de acompanhamento, você tem permissão para fazer isso.\nQuando solicitado a comparar informações (ex: qual é mais quente, maior, etc.), use as informações do histórico da conversa e dos resultados das ferramentas.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Como está o tempo em São Paulo hoje?', additional_kwargs={}, response_metadata={}), AIMessage(content='Preciso de mais informações para responder à sua pergunta. Você poderia me dizer a data de hoje?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safet

In [9]:
messages = [HumanMessage(content="Qual está mais quente?")]
thread = {"configurable": {"thread_id": "2"}} 

print("\n--- Pergunta 4: Comparação ---")
for event in abot.graph.stream({"messages": messages}, thread):
    for k, v in event.items():
        if k in ("llm", "action"):
            print(f"{k}: {v['messages']}")


--- Pergunta 4: Comparação ---
Mensagens enviadas ao modelo: [SystemMessage(content='Você é um assistente de pesquisa inteligente. Use o mecanismo de busca (tavily_search_results_json) para procurar informações.\nVocê tem permissão para fazer múltiplas chamadas à ferramenta (em conjunto ou em sequência).\nBusque informações apenas quando tiver certeza do que procurar.\nSe precisar de mais detalhes para formular uma pergunta de acompanhamento, você tem permissão para fazer isso.\nQuando solicitado a comparar informações (ex: qual é mais quente, maior, etc.), use as informações do histórico da conversa e dos resultados das ferramentas.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Qual está mais quente?', additional_kwargs={}, response_metadata={}), AIMessage(content='Mais quente do que o quê? Por favor, forneça o contexto para que eu possa responder à sua pergunta.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings':